# Scope of Notebook

The goal of this notebook is to showcase how you can prepare data for the future goal of consumption by an ML model, and leveraging functionality in the Adobe Experience Platform to generate features at scale and make it available in your choice of cloud storage.

![Workflow](../media/CMLE-SageMaker-Notebooks-Week2-Workflow.png)

We'll go through several steps:
- **Creating a query** to encapsulate what a good featurized dataset will be.
- **Executing that query** and storing the results.
- **Setting up a flow** to export the results into cloud storage.
- **Executing that flow** to deliver the results.

# Setup

This notebook requires some configuration data to properly authenticate to your Adobe Experience Platform instance. You should be able to find all the values required above by following the Setup section of the **README**.

The next cell will be looking for your configuration file under your **ADOBE_HOME** path to fetch the values used throughout this notebook. See more details in the Setup section of the **README** to understand how to create your configuration file.

In [ ]:
import os
from configparser import ConfigParser

os.environ["ADOBE_HOME"] = "../../"
if "ADOBE_HOME" not in os.environ:
    raise Exception("ADOBE_HOME environment variable needs to be set.")

config = ConfigParser()
config_path = os.path.join(os.environ["ADOBE_HOME"], "conf", config_file)

if not os.path.exists(config_path):
    raise Exception(f"Looking for configuration under {config_path} but config not found, please verify path")

config.read(config_path)

ims_org_id = config.get("Platform", "ims_org_id")
sandbox_name = config.get("Platform", "sandbox_name")
environment = config.get("Platform", "environment")
client_id = config.get("Authentication", "client_id")
client_secret = config.get("Authentication", "client_secret")
scopes = config.get("Authentication", "scopes")
tech_account_id = config.get("Authentication", "tech_acct_id")
dataset_id = config.get("Platform", "dataset_id")
if not dataset_id:
    raise Exception("Looking for dataset_id saved in week1 notebook to continue the exploration in week2 notebook but not found")
export_path = config.get("Cloud", "export_path")
data_format = config.get("Cloud", "data_format")
compression_type = config.get("Cloud", "compression_type")

s3_bucket_name = config.get("AWS","s3_bucket_name")
s3_prefix = config.get("AWS","s3_prefix")
cfn_stack_id = config.get("AWS", "cfn_stack_id")

if not s3_bucket_name or not s3_prefix or not cfn_stack_id:
    raise Exception("Please make sure the above fields s3_bucket_name, s3_prefix, cfn_stack_id are all populated with valid values in config.ini under the AWS section")

Some utility functions that will be used throughout this notebook:

In [ ]:
import json
NOTEBOOK_METADATA_FILE="/opt/ml/metadata/resource-metadata.json"
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE,"rb") as f:
        username = json.loads(f.read())['UserProfileName']
print(f"Username: {username}")

In [2]:
def get_ui_link(tenant_id, resource_type, resource_id):
    if environment == "prod":
        prefix = f"https://experience.adobe.com"
    else:
        prefix = f"https://experience-{environment}.adobe.com"
    return f"{prefix}/#/@{tenant_id}/sname:{sandbox_name}/platform/{resource_type}/{resource_id}"

Before we run anything, make sure to install the following required libraries for this notebook. They are all publicly available libraries and the latest version should work fine.

<div class="alert alert-block alert-warning">
<b>Note:</b> 
    
If you are not using `pip` to manage Python packages, or that you use a variation like `pip3`, please update the cell below accordingly to install these packages.
</div>

In [3]:
!pip install aepp
!pip install pygresql==5.2.5

We'll be using the [aepp Python library](https://github.com/pitchmuc/aepp) here to interact with AEP APIs and create a schema and dataset suitable for adding our synthetic data further down the line. This library simply provides a programmatic interface around the REST APIs, but all these steps could be completed similarly using the raw APIs directly or even in the UI. For more information on the underlying APIs please see [the API reference guide](https://developer.adobe.com/experience-platform-apis/).

Before any calls can take place, we need to configure the library and setup authentication credentials. For this you'll need the following piece of information. For information about how you can get these, please refer to the `Setup` section of the **README**:
- Client ID
- Client secret
- Private key
- Technical account ID

In [3]:
import aepp

aepp.configure(
  org_id=ims_org_id,
  tech_id=tech_account_id, 
  secret=client_secret,
  scopes=scopes,
  client_id=client_id,
  sandbox=sandbox_name,
  environment=environment
)

# 1. Creating Featurization Query

In the previous week we created some synthetic data under a dataset in your Adobe Experience Platform instance, and now we're ready to use it to generate features that can then be fed to our ML model. For this purpose we'll be using the [Query Service](https://experienceleague.adobe.com/docs/experience-platform/query/home.html?lang=en#) which lets us access data from any dataset and run queries at scale. The end goal here is to compress this dataset into a small subset of meaningful features that will be relevant to our model.

Before we can issue queries, we need to find the table name corresponding to our dataset. Please make sure your dataset ID was entered in your configuration as part of the setup, it should be available under the `dataset_id` variable in this notebook.

Every dataset in AEP should have a corresponding table in PQS based on its name. We can use AEP APIs to get that information with the code below:

In [4]:
from aepp import catalog

cat_conn = catalog.Catalog()

dataset_info = cat_conn.getDataSet(dataset_id)
table_name = dataset_info[dataset_id]["tags"]["adobe/pqs/table"][0]
table_name

'cmle_week1_new_notebook_with_ecid_used_dataset_created_by_saikat'

And because some of the data we created in the previous week is under a custom field group, we need to fetch your tenant ID since the data will be nested under it. This can be accomplished simply with the code below

In [5]:
from aepp import schema

schema_conn = schema.Schema()

tenant_id = schema_conn.getTenantId()
tenant_id

'aamds'

Now we can use that table to query it via Query Service. Every query is able to run at scale leveraging Spark-based distributed computing power in the backend, so the goal is to take this large dataset, extract meaningful features and only keep a smaller subset to feed into a ML model.

We'll be leveraging `aepp` again to interact with Query Service.

In [6]:
from aepp import queryservice

qs = queryservice.QueryService()
qs_conn = qs.connection()
qs_conn['dbname']=f'{sandbox_name}:{table_name}'
qs_cursor = queryservice.InteractiveQuery(qs_conn)

<div class="alert alert-block alert-warning">
<b>Note:</b> If at any point in this notebook the connection to Query Service is closed, you can refresh it by re-enabling that cell.
</div>

Let's first define our ML problem scientifically:
- **What kind of problem** are we solving? We'd like to predict whether someone is likely to subscribe or not. We treat that as a binary classification problem, you are either subscribed or you are not.
- What is our **target variable**? Whether a subscription occurred or not for a given user.
- What are our **positive labels**? People who have at least 1 event corresponding to a subscription (marked as `web.formFilledOut`). We will keep a single feature row from the event where they actually subscribed.
- What are our **negative labels**? People who don't have a single event corresponding to a subscription. We will keep a random row to avoid having bias in the data.

Let's put that into practice and start looking at our positive labels:

In [7]:
query_positive_labels = f"""
SELECT *
FROM (
    SELECT
        eventType,
        _{tenant_id}.cmle_id as userId,
        SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY _{tenant_id}.cmle_id) 
            AS "subscriptionOccurred"
    FROM {table_name}
)
WHERE subscriptionOccurred = 1 AND eventType = 'web.formFilledOut'
"""

df_positive_labels = qs_cursor.query(query_positive_labels, output="dataframe")
print(f"Number of positive classes: {len(df_positive_labels)}")
df_positive_labels.head()

Number of positive classes: 15030


,eventType,userId,subscriptionOccurred
0,web.formFilledOut,01054714817856066632264746967668888198,1
1,web.formFilledOut,01108987448326166452882902459550847869,1
2,web.formFilledOut,01234932524049684301600430771576676651,1
3,web.formFilledOut,01242509742845154266705851412711275265,1
4,web.formFilledOut,01370262440242301894771514187123049829,1


<div class="alert alert-block alert-warning">
<b>Note:</b> 
    
We are using a sub-query because we want to filter on the `subscriptionOccurred` which is defined as part of the query, so it can't be used in a filter condition unless it is in a sub-query.
</div>

Now let's look at our negative labels. Because we just want to retain a random row to avoid bias, we need to introduce randomness into our query:

In [8]:
query_negative_labels = f"""
SELECT *
FROM (
    SELECT
        _{tenant_id}.cmle_id as userId,
        SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY _{tenant_id}.cmle_id) 
            AS "subscriptionOccurred",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
)
WHERE subscriptionOccurred = 0 AND random_row_number_for_user = 1
"""

df_negative_labels = qs_cursor.query(query_negative_labels, output="dataframe")
print(f"Number of negative classes: {len(df_negative_labels)}")
df_negative_labels.head()

Number of negative classes: 50000


,userId,subscriptionOccurred,random_row_number_for_user
0,01027994177972439148069092698714414382,0,1
1,01066367784525869370502097766291450913,0,1
2,01117296890525140996735553609305695042,0,1
3,01149554820363915324573708359099551093,0,1
4,01172121447143590196349410086995740317,0,1


Putting it all together, we can query both our positive and negative classes with the following query:

In [9]:
query_labels = f"""
SELECT *
FROM (
    SELECT
        eventType,
        _{tenant_id}.cmle_id as userId,
        SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY _{tenant_id}.cmle_id) 
            AS "subscriptionOccurred",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
"""

df_labels = qs_cursor.query(query_labels, output="dataframe")
print(f"Number of classes: {len(df_labels)}")
df_labels.head()

Number of classes: 50000


,eventType,userId,subscriptionOccurred,random_row_number_for_user
0,commerce.productViews,01027994177972439148069092698714414382,0,1
1,web.formFilledOut,01054714817856066632264746967668888198,1,5
2,web.formFilledOut,01108987448326166452882902459550847869,1,1
3,directMarketing.emailSent,01117296890525140996735553609305695042,0,1
4,directMarketing.emailOpened,01149554820363915324573708359099551093,0,1


Now let's think what kind of features make sense for this kind of problem that we would like to eventually feed to an ML model. There's 2 main kinds of features we're interested in:
- **Actionable features**: different actions the user actually took in response to a marketing event.
- **Temporal features**: distribution over time of the actions the user took. This is useful for modeling to capture behavior over time and not just statically at any particular point in time.

We can think of a few simple things that can be derived from this data for the actionable features:
- **Number of emails** that were sent for marketing purposes and received by the user.
- Portion of these emails that were actually **opened**.
- Portion of these emails where the user actually **clicked** on the link.
- **Number of products** that were viewed.
- Number of **propositions that were interacted with**.
- Number of **propositions that were dismissed**.
- Number of **links that were clicked on**.

Regarding the temporal features, we can look at consecutive occurrences between various events. This can be accomplished using the `TIME_BETWEEN_PREVIOUS_MATCH` function. So we take the previous actionable features and look at their temporal distribution:
- Number of minutes between 2 consecutive emails received.
- Number of minutes between 2 consecutive emails opened.
- Number of minutes between 2 consecutive emails where the user actually clicked on the link.
- Number of minutes between 2 consecutive product views.
- Number of minutes between 2 propositions that were interacted with.
- Number of minutes between 2 propositions that were dismissed.
- Number of minutes between 2 links that were clicked on.

Let's put all that in practice and look how we can create these features inside a query:

In [10]:
query_features = f"""
SELECT
    _{tenant_id}.cmle_id as userId,
    SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "emailsReceived",
    SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "emailsOpened",       
    SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "emailsClicked",       
    SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "productsViewed",       
    SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "propositionInteracts",       
    SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "propositionDismissed",
    SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "webLinkClicks" ,
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_emailSent",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_emailOpened",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_emailClick",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_productView",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_propositionInteract",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_propositionDismiss",
    TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
       OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
       AS "minutes_since_linkClick"
FROM {table_name}
"""

df_features = qs_cursor.query(query_features, output="dataframe")
df_features.head()

,userId,emailsReceived,emailsOpened,emailsClicked,productsViewed,propositionInteracts,propositionDismissed,webLinkClicks,minutes_since_emailSent,minutes_since_emailOpened,minutes_since_emailClick,minutes_since_productView,minutes_since_propositionInteract,minutes_since_propositionDismiss,minutes_since_linkClick
0,01046464403641925015932682550398209571,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,01046464403641925015932682550398209571,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN
2,01046464403641925015932682550398209571,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN
3,01046464403641925015932682550398209571,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,None,0.0
4,01046464403641925015932682550398209571,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,None,1.0


At that point we have defined all our features, and we also have our classes cleanly defined, so we can tie everything together in a final query that will represent our training set to be used later on on our ML model.

In [11]:
query_training_set = f"""
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name} LIMIT 1000
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
ORDER BY timestamp
"""

df_training_set = qs_cursor.query(query_training_set, output="dataframe")
df_training_set.head()

,userId,eventType,timestamp,subscriptionOccurred,emailsReceived,emailsOpened,emailsClicked,productsViewed,propositionInteracts,propositionDismissed,webLinkClicks,minutes_since_emailSent,minutes_since_emailOpened,minutes_since_emailClick,minutes_since_productView,minutes_since_propositionInteract,minutes_since_propositionDismiss,minutes_since_linkClick,random_row_number_for_user
0,01024918979328021024268398155289293841,advertising.impressions,2023-01-11 11:01:28.844,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN,1
1,01313428821394797049107854074567613407,directMarketing.emailSent,2023-01-11 11:07:12.511,0,1,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN,None,NaN,1
2,02993793241626222851527839833265197633,web.formFilledOut,2023-01-11 15:36:21.667,1,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,1.0,None,NaN,10
3,03284744679335392483733573117571851574,advertising.impressions,2023-01-11 19:53:50.963,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,None,NaN,1
4,03077253005436651532049293706745952081,directMarketing.emailOpened,2023-01-11 20:45:29.123,0,1,3,0,0,0,0,0,829.0,0.0,NaN,NaN,NaN,None,NaN,1


# 2. Generating Features Incrementally

Now in a typical ML workload you'll want to use incremental data to feed to your model, or data between some specific dates. For that purpose we can use snapshot information that is tracked inside Query Service every time a new batch of data is ingested, using the `history_meta` metadata table. For example, you can access the metadata for each batch of your dataset using the query below:

In [12]:
query_meta = f"""
SELECT * FROM (SELECT history_meta('{table_name}'))
"""

df_meta = qs_cursor.query(query_meta, output="dataframe")
print(f"Total number of snapshots/batches: {len(df_meta)}")
df_meta.head(n=10)

Total number of snapshots/batches: 10


,snapshot_generation,made_current_at,snapshot_id,parent_id,is_current_ancestor,is_current,output_record_count,output_byte_size
0,0,2023-04-04 15:44:44.955,3825256663640337857,NaN,True,False,106317,14641809
1,1,2023-04-04 15:44:46.256,5250805769364495228,3.825257e+18,True,False,106736,14741300
2,2,2023-04-04 15:44:48.604,8316771932049439874,5.250806e+18,True,False,106059,14683324
3,3,2023-04-04 15:44:50.096,7241083319934178488,8.316772e+18,True,False,106474,14697877
4,4,2023-04-04 15:44:52.253,3157125847948566759,7.241083e+18,True,False,106675,14725877
5,5,2023-04-04 15:44:59.995,4460330323826197864,3.157126e+18,True,False,106716,14734401
6,6,2023-04-04 15:45:19.237,4818579672832050765,4.460330e+18,True,False,106525,14711442
7,7,2023-04-04 15:45:31.781,7870632534228575321,4.818580e+18,True,False,106576,14750647
8,8,2023-04-04 15:45:43.676,1776236004746194930,7.870633e+18,True,False,107138,14766853
9,9,2023-04-04 15:45:44.813,6433588580290892078,1.776236e+18,True,True,107056,14779922


Now let's use that information to transform our featurization query into an incremental version of it. We can use [anonymous blocks](https://experienceleague.adobe.com/docs/experience-platform/query/sql/syntax.html?lang=en#anonymous-block) to create variables used to filter on the snapshots. Anonymous blocks are useful to embed multiple queries at once and do things like defining variables and such. We can then use Query Service's `SNAPSHOT BETWEEN x AND y` functionality to query data incrementally.

In our case because this is the first time generating the features we will look at data between the most recent snapshot and its preceding one (which should correspond to the last batch of data ingested), but this can be extended to query data between any snapshots:

In [13]:
f"""
$$ BEGIN

SET @from_snapshot_id = SELECT parent_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;
SET @to_snapshot_id = SELECT snapshot_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;

SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN @from_snapshot_id AND @to_snapshot_id
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
ORDER BY timestamp;

EXCEPTION
  WHEN OTHER THEN
    SELECT 'ERROR';

END $$;
"""

'\n$$ BEGIN\n\nSET @from_snapshot_id = SELECT parent_id FROM (SELECT history_meta(\'cmle_week1_new_notebook_with_ecid_used_dataset_created_by_saikat\')) WHERE is_current = true;\nSET @to_snapshot_id = SELECT snapshot_id FROM (SELECT history_meta(\'cmle_week1_new_notebook_with_ecid_used_dataset_created_by_saikat\')) WHERE is_current = true;\n\nSELECT *\nFROM (\n    SELECT _aamds.userid as userId, \n       eventType,\n       timestamp,\n       SUM(CASE WHEN eventType=\'web.formFilledOut\' THEN 1 ELSE 0 END) \n           OVER (PARTITION BY _aamds.userid) \n           AS "subscriptionOccurred",\n       SUM(CASE WHEN eventType=\'directMarketing.emailSent\' THEN 1 ELSE 0 END) \n           OVER (PARTITION BY _aamds.userid ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) \n           AS "emailsReceived",\n       SUM(CASE WHEN eventType=\'directMarketing.emailOpened\' THEN 1 ELSE 0 END) \n           OVER (PARTITION BY _aamds.userid ORDER BY timestamp ROWS BETWEEN UNBOUNDED P

Note that we're not executing it interactively because this anonymous block is actually multiple queries chained together, and there is no simple way to return multiple result sets using PostgreSQL libraries.

To solve that we can executed the query asynchronously and add a `CREATE TABLE x AS` statement at the beginning of our featurization query, to then look in that table. Note that because this is executed asynchronously, it goes into the Query Service scheduler and will take a few minutes to start executing, unlike the code we've been running until now which was synchronous and instant.

In [14]:
import time
import sys

ctas_table_name = f"cmle_example_training_set_incremental_{username}"

query_training_set_incremental = f"""
$$ BEGIN

SET @from_snapshot_id = SELECT parent_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;
SET @to_snapshot_id = SELECT snapshot_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;

CREATE TABLE {ctas_table_name} AS
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN @from_snapshot_id AND @to_snapshot_id LIMIT 1000
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
ORDER BY timestamp;

EXCEPTION
  WHEN OTHER THEN
    SELECT 'ERROR';

END $$;
"""

query_incremental_res = qs.postQueries(
    name="[CMLE][Week2] Query to generate incremental training data",
    sql=query_training_set_incremental,
    dbname=f"{sandbox_name}:{table_name}"
)
query_incremental_id = query_incremental_res["id"]
print(f"Query started successfully and got assigned ID {query_incremental_id} - it will take some time to execute")

Query started successfully and got assigned ID dc46636d-5c19-40f7-a772-524cc238a93e - it will take some time to execute


In [15]:
def wait_for_query_completion(query_id):
    while True:
        query_info = qs.getQuery(query_id)
        query_state = query_info["state"]
        if query_state in ["SUCCESS", "FAILED"]:
            break
        print("Query is still in progress, sleeping...")
        time.sleep(60)

    duration_secs = query_info["elapsedTime"] / 1000
    if query_state == "SUCCESS":
        print(f"Query completed successfully in {duration_secs} seconds")
    else:
        print(f"Query failed with the following errors:", file=sys.stderr)
        for error in query_info["errors"]:
            print(f"Error code {error['code']}: {error['message']}", file=sys.stderr)

In [16]:
wait_for_query_completion(query_incremental_id)

Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query completed successfully in 386.37 seconds


<div class="alert alert-block alert-warning">
<b>Note:</b> This should run in less than 10 minutes. If for whatever reason this does not finish in that time frame, it may get stuck creating the batch if the ingestion service is busy ingesting other data in your organization. We advise waiting longer or reaching out to your Adobe contact if this does not complete.
    
The same comment applies to subsequent cells where we use `wait_for_query_completion`.
</div>

This `CREATE TABLE x AS` statement actually does several steps:
- It will create a brand **new dataset** in your Adobe Experience Platform organization and sandbox.
- The **schema** for this dataset will be created ad-hoc to **match the fields** of our featurization query, so there is no need to manually create schemas and fieldgroups for this.

You can verify that by going in the UI at the link below, and making sure it you see the dataset as shown in the screenshot further down. The number of records should correspond to the batch size you used in the previous week since we are querying a single snapshot/batch.

<div class="alert alert-block alert-warning">
<b>Note:</b> Re-executing the same query will fail, because it will try to create a table and not insert into it. We're solving this problem in the next section.
</div>

In [17]:
datasets_res = cat_conn.getDataSets(name=ctas_table_name)
if len(datasets_res) != 1:
    raise Exception(f"Expected a single dataset but got {len(datasets_res)} ones")
ctas_dataset_id = list(datasets_res.keys())[0]
ctas_dataset_link = get_ui_link(tenant_id, "dataset/browse", ctas_dataset_id)
print(f"Dataset available as dataset ID {ctas_dataset_id} under {ctas_dataset_link}")

Dataset available as dataset ID 642d8e01ec08531c065b48b2 under https://experience.adobe.com/#/@aamds/sname:cmle/platform/dataset/browse/642d8e01ec08531c065b48b2


![CTAS](../media/CMLE-Notebooks-Week2-CTAS.png)

Now we can just query it to see the structure of the data and verify it matches our query:

In [18]:
query_ctas = f"""
SELECT * FROM {ctas_table_name} LIMIT 10;
"""
qs = queryservice.QueryService()
qs_conn = qs.connection()
qs_conn['dbname']=f'{sandbox_name}:{ctas_table_name}'
qs_cursor = queryservice.InteractiveQuery(qs_conn)
df_ctas = qs_cursor.query(query_ctas, output="dataframe")
df_ctas.head()

,userId,eventType,timestamp,subscriptionOccurred,emailsReceived,emailsOpened,emailsClicked,productsViewed,propositionInteracts,propositionDismissed,webLinkClicks,minutes_since_emailSent,minutes_since_emailOpened,minutes_since_emailClick,minutes_since_productView,minutes_since_propositionInteract,minutes_since_propositionDismiss,minutes_since_linkClick,random_row_number_for_user
0,03496398227344712440123581041340192961,web.webpagedetails.pageViews,2023-01-10 14:44:38.161,0,0,0,0,0,0,0,0,NaN,NaN,None,None,None,None,NaN,1
1,01306047604382086256919152660635616402,directMarketing.emailSent,2023-01-10 23:28:31.161,0,1,0,0,0,0,0,0,0.0,NaN,None,None,None,None,NaN,1
2,01899181931701313368403223253046030321,directMarketing.emailSent,2023-01-12 09:54:47.161,0,1,0,0,0,0,0,1,0.0,NaN,None,None,None,None,404.0,1
3,03077253005436651532049293706745952081,directMarketing.emailSent,2023-01-12 10:15:25.161,0,2,3,0,0,0,0,0,0.0,809.0,None,None,None,None,NaN,1
4,03622335536366276762200832126146857535,directMarketing.emailSent,2023-01-12 23:25:00.161,0,1,0,0,0,0,0,0,0.0,NaN,None,None,None,None,NaN,1


# 3. Templatizing the Featurization Query

Now we've got a complete featurization query that can also be used to generate features incrementally, but we still want to go further:
- The **snapshot window should be configurable** and easy to change without having to constantly create new queries.
- The query itself should be **stored in a templatized way** so it can be referred to easily.
- The query should be able to **create the table** automatically as well as **inserting into** a pre-existing table.

Query Service has this concept of [templates](https://experienceleague.adobe.com/docs/experience-platform/query/ui/query-templates.html?lang=en) that we will be leveraging in this section to satisfy the requirements mentioned above.

The first step is to make make sure we either create the table if it does not exist, otherwise insert into it. This can be done by checking if the table exists using the `table_exists` function and adding a condition in our anonymous block based on that:

In [19]:
query_training_set_ctas_or_insert = f"""
$$ BEGIN

SET @from_snapshot_id = SELECT parent_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;
SET @to_snapshot_id = SELECT snapshot_id FROM (SELECT history_meta('{table_name}')) WHERE is_current = true;
SET @my_table_exists = SELECT table_exists('{ctas_table_name}');

CREATE TABLE IF NOT EXISTS {ctas_table_name} AS
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN @from_snapshot_id AND @to_snapshot_id LIMIT 1000
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
ORDER BY timestamp;

INSERT INTO {ctas_table_name}
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN @from_snapshot_id AND @to_snapshot_id LIMIT 1000
)
WHERE 
    @my_table_exists = 't' AND
    ((subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1))
ORDER BY timestamp;

EXCEPTION
  WHEN OTHER THEN
    SELECT 'ERROR';

END $$;
"""

query_ctas_or_insert_res = qs.postQueries(
    name="[CMLE][Week2] Query to generate training data as CTAS or Insert",
    sql=query_training_set_ctas_or_insert,
    dbname=f"{sandbox_name}:all"
)
query_ctas_or_insert_id = query_ctas_or_insert_res["id"]
print(f"Query started successfully and got assigned ID {query_ctas_or_insert_id} - it will take some time to execute")

wait_for_query_completion(query_ctas_or_insert_id)

Query started successfully and got assigned ID f7fd9bd5-cc7d-4984-b33a-f639cd48fd18 - it will take some time to execute
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query completed successfully in 202.07 seconds


The next step is to make the snapshot time window configurable. To do that we can replace the part containing the snapshot boundaries with variables as `$variable` so they can be passed at runtime using Query Service:

In [20]:
ctas_table_name = f"cmle_training_set_{username}"

query_training_set_template = f"""
$$ BEGIN

SET @my_table_exists = SELECT table_exists('{ctas_table_name}');

CREATE TABLE IF NOT EXISTS {ctas_table_name} AS
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN $from_snapshot_id AND $to_snapshot_id
)
WHERE (subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1)
ORDER BY timestamp;

INSERT INTO {ctas_table_name}
SELECT *
FROM (
    SELECT _{tenant_id}.cmle_id as userId, 
       eventType,
       timestamp,
       SUM(CASE WHEN eventType='web.formFilledOut' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id) 
           AS "subscriptionOccurred",
       SUM(CASE WHEN eventType='directMarketing.emailSent' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsReceived",
       SUM(CASE WHEN eventType='directMarketing.emailOpened' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsOpened",       
       SUM(CASE WHEN eventType='directMarketing.emailClicked' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "emailsClicked",       
       SUM(CASE WHEN eventType='commerce.productViews' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "productsViewed",       
       SUM(CASE WHEN eventType='decisioning.propositionInteract' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionInteracts",       
       SUM(CASE WHEN eventType='decisioning.propositionDismiss' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "propositionDismissed",
       SUM(CASE WHEN eventType='web.webinteraction.linkClicks' THEN 1 ELSE 0 END) 
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "webLinkClicks" ,
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailSent', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailSent",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailOpened', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailOpened",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'directMarketing.emailClicked', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_emailClick",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'commerce.productViews', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_productView",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'decisioning.propositionInteract', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionInteract",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'propositionDismiss', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_propositionDismiss",
       TIME_BETWEEN_PREVIOUS_MATCH(timestamp, eventType = 'web.webinteraction.linkClicks', 'minutes')
           OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
           AS "minutes_since_linkClick",
        row_number() OVER (PARTITION BY _{tenant_id}.cmle_id ORDER BY randn()) AS random_row_number_for_user
    FROM {table_name}
    SNAPSHOT BETWEEN $from_snapshot_id AND $to_snapshot_id
)
WHERE 
    @my_table_exists = 't' AND
    ((subscriptionOccurred = 1 AND eventType = 'web.formFilledOut') OR (subscriptionOccurred = 0 AND random_row_number_for_user = 1))
ORDER BY timestamp;

EXCEPTION
  WHEN OTHER THEN
    SELECT 'ERROR';

END $$;
"""

We're not executing it because it has actual variables in it that will need to be resolved at runtime, so executing it right now would fail. We're ready to turn this into a proper template, which requires the following:
- A **name** for your templatized query.
- Some set of **query parameters** that you might want to already save - in our case we're not setting any so both snapshot boundaries can be set at runtie.
- Your SQL **query**.

Once you do this, the template should be available in the UI at the link below, as you can see in the screenshot.

In [21]:
template_res = qs.createQueryTemplate({
  "sql": query_training_set_template,
  "queryParameters": {},
  "name": f"[CMLE][Week2] Template for training data created by {username}"
})
template_id = template_res["id"]
template_link = get_ui_link(tenant_id, "query/edit", template_id)

print(f"Query template for training data created as ID {template_id} under {template_link}")

Query template for training data created as ID f1d7db3a-adcb-472b-ac7d-c91b355b1182 under https://experience.adobe.com/#/@aamds/sname:cmle/platform/query/edit/f1d7db3a-adcb-472b-ac7d-c91b355b1182


![Template](../media/CMLE-Notebooks-Week2-Template.png)

Now that the template is saved, we can refer to it at any time, and passing any kind of values for the snapshots that we want. So for example if you have streaming data coming through your system, you just need to find out the beginning snapshot ID and end snapshot ID, and you can execute this featurization query that will take care of querying between these 2 snapshots.

In this example, we'll just query the entire dataset, so the very first and very last snapshots:

In [22]:
query_snapshots = f"""
SELECT snapshot_id 
FROM (
    SELECT history_meta('{table_name}')
) 
WHERE is_current = true OR snapshot_generation = 0 
ORDER BY snapshot_generation ASC
"""

df_snapshots = qs_cursor.query(query_snapshots, output="dataframe")

snapshot_start_id = str(df_snapshots["snapshot_id"].iloc[0])
snapshot_end_id = str(df_snapshots["snapshot_id"].iloc[1])
print(f"Query will go from start snapshot ID {snapshot_start_id} to end snapshot ID {snapshot_end_id}")

df_snapshots.head()

Query will go from start snapshot ID 3825256663640337857 to end snapshot ID 6433588580290892078


,snapshot_id
0,3825256663640337857
1,6433588580290892078


In [23]:
query_final_res = qs.postQueries(
    name=f"[CMLE][Week2] Query to generate training data created by {username}",
    templateId=template_id,
    queryParameters={
        "from_snapshot_id": snapshot_start_id,
        "to_snapshot_id": snapshot_end_id,
    },
    dbname=f"{sandbox_name}:all"
)
query_final_id = query_final_res["id"]
print(f"Query started successfully and got assigned ID {query_final_id} - it will take some time to execute")

wait_for_query_completion(query_final_id)

Query started successfully and got assigned ID cf117097-703c-4851-957b-8d8571119248 - it will take some time to execute
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query is still in progress, sleeping...
Query completed successfully in 284.399 seconds


At that point we got our full featurized dataset that is ready to plug into a ML model. But it's still in an Adobe Experience Platform dataset so far, and we need to bring it back to our cloud storage account outside of the Experience Platform to use our tool of choice, which will be covered in the next section.

Before we go through that, we just need to find te dataset ID corresponding to the output of our templatized query:

In [24]:
ctas_table_info = cat_conn.getDataSets(name=ctas_table_name)
created_dataset_id = list(ctas_table_info.keys())[0]
created_dataset_link = get_ui_link(tenant_id,"dataset/browse",created_dataset_id)
print(f"Featurized data available in table {ctas_table_name} under dataset ID {created_dataset_id} under {created_dataset_link}")

Featurized data available in table cmle_training_set_cmenguy under dataset ID 642d912512f86b1c07ef5650 under https://experience.adobe.com/#/@aamds/sname:cmle/platform/dataset/browse/642d912512f86b1c07ef5650


# 4. Exporting the Featurized Dataset to Cloud Storage

Now that our featurized data is in a dataset, we need to bring it out to an external cloud storage filesystem from which the ML model training and scoring will be performed. 

For the purposes of this notebook we will be using the [Amazon S3](https://experienceleague.adobe.com/docs/experience-platform/destinations/catalog/cloud-storage/amazon-s3.html?lang=en#changelog) as the filesystem. Every Adobe Experience Platform has a S3 already setup as an [Amazon S3](https://aws.amazon.com/s3/) container. We'll be using that as a delivery mechanism for the featurized data, but this step can be customized to delivery this data to any cloud storage filesystem.

To setup the delivery pipeline, we'll be using the [Flow Service for Destinations](https://developer.adobe.com/experience-platform-apis/references/destinations/) which will be responsible for picking up the featurized data and dump it into the S3. There's a few steps involved:
- Creating a **source connection**.
- Creating a **target connection**.
- Creating a **data flow**.

For that, again we use `aepp` to abstract all the APIs:

In [25]:
from aepp import flowservice

flow_conn = flowservice.FlowService()

## 4.1 Creating the Source Connection

The source connection is responsible for connecting to your Adobe Experience Platform dataset so that the resulting flow will know exactly where to look for the data and in what format.

In [26]:
s3_source_connection = {
            "name": f"[CMLE][Week2] Featurized Dataset S3 source connection created by {username}",
            "data": {
                "format": "parquet"
            },
            "connectionSpec": {
                "id": "23598e46-f560-407b-88d5-ea6207e49db0",
                "version": "1.0"
            },
            "params": {
                "datasets": [{"dataSetId": created_dataset_id}]
            }
        }
        
source_res = flow_conn.createSourceConnection(
    data = s3_source_connection
)
source_connection_id = source_res["id"]
source_connection_id

'8858421d-1f92-4787-a140-caacd179ad9f'

## 4.2 Creating the Base and Target Connection

The target connection is responsible for connecting to the destination filesystem. In our case, we want to connect to the DLZ and specify in what format the data will be stored, as well as the type of compression.

Before we can create it however, we need to create a base connection to the Amazon S3. A base connection is just an instance of a connection spec that details how one authenticates to a particular destination. In our case, because we're using the S3 which is a known entity internal to Adobe, we can just reference the standard S3 connection spec ID and create an empty base connection.

For reference, here is a list of all the connection specs available for the most popular cloud storage accounts (these IDs are global across every single customer account and sandbox):

| Cloud Storage Type    | Connection Spec ID                   |
|-----------------------|--------------------------------------|
| Amazon S3             | 4fce964d-3f37-408f-9778-e597338a21ee |
| Azure Blob Storage    | 6d6b59bf-fb58-4107-9064-4d246c0e5bb2 |
| Azure Data Lake       | be2c3209-53bc-47e7-ab25-145db8b873e1 |
| Data Landing Zone     | 10440537-2a7b-4583-ac39-ed38d4b848e8 |
| Google Cloud Storage  | c5d93acb-ea8b-4b14-8f53-02138444ae99 |
| SFTP                  | 36965a81-b1c6-401b-99f8-22508f1e6a26 |

When creating the base connection, an auth attr expects a static set of AWS key/secret.This gets stored in a secrets service managed by Adobe. The CDK deployment created an IAMuser with an access key and secret key specifically for this base connection.

In [ ]:
import boto3
from botocore.exceptions import ValidationError, ClientError
cfn = boto3.client('cloudformation')
secrets_manager = boto3.client('secretsmanager')
try:
    # validate cloudformation ID and get outputs
    print('Validating CloudFormation ID')
    response = cfn.describe_stacks(StackName=cfn_stack_id)
    print('Stack found')
    outputs = response['Stacks'][0]['Outputs']
    for output in outputs:
        if output['OutputKey'] == 'DataFlowUserAccessKey':
            access_key = output['OutputValue']
            print(f'Found access key: {access_key}')
        if output['OutputKey'] == 'DataFlowUserSecretKey':
            secret_name = output['OutputValue']
            print(f'Found secret stored in Secrets Manager: {secret_name}')
except ValidationError as e:
    print(f'Could not find stack from provided stack ID: {cfn_stack_id}')
except ClientError as e:
    raise(e)
    
try:
    response = secrets_manager.get_secret_value(SecretId=secret_name)
    secret_key = response['SecretString']
except ClientError as e:
    raise(e)
    
print(f"ACCESS: {access_key}")
print(f"SECRET: {secret_key}")

In [27]:
base_connection_res = flow_conn.createConnection(data={
    "name": f"[CMLE][Week2] Base Connection to S3 created by {username}",
    "auth": {
      "specName": "Access Key",
      "params": {
        "s3SecretKey": secret_key,
        "s3AccessKey": access_key,
        
      }
    },
    "connectionSpec": {
        "id": "4fce964d-3f37-408f-9778-e597338a21ee",
        "version": "1.0"
    }
}) 
base_connection_id = base_connection_res["id"]

'a8cc0b95-590f-4fa6-8b95-65a58774271f'

With that base connection, we're ready to create the target connection that will tie to our S3 directory: Please replace the bucketName and the path with valid values from your S3 configs

In [28]:
# TODO: implement in aepp a way to abstract that
target_res = flow_conn.createTargetConnection(
    data={
        "name": f"[CMLE][Week2] S3 target connection created by {username}",
        "baseConnectionId": base_connection_id,
        "params": {
            "mode": "Server-to-server",
            "compression": compression_type,
            "datasetFileType": data_format,
            "bucketName": s3_bucket_name,
            "path": s3_prefix
        },
        "connectionSpec": {
            "id": "4fce964d-3f37-408f-9778-e597338a21ee",
            "version": "1.0"
        }
    }
)
target_connection_id = target_res["id"]

'7092cdbf-23e4-44f1-be45-79ff03c42ce9'

<div class="alert alert-block alert-warning">
<b>Note:</b> 
    
If you would like to switch to a different cloud storage, you need to update the `connection_spec_id` variable above to the matching value in the table mentioned earlier in this section.
</div>

## 4.3 Creating the Data Flow

Now that we have the source and target connections setup, we can construct the data flow. A data flow is the "recipe" that describes where the data comes from and where it should end up. We can also specify how often checks happen to find new data, but it cannot be lower than 3 hours currently for platform stability reasons. A data flow is tied to a flow spec ID which contains the instructions for transfering data in an optimized way between a source and destination.

For reference, here is a list of all the flow specs available for the most popular cloud storage accounts (these IDs are global across every single customer account and sandbox):

| Cloud Storage Type    | Flow Spec ID                         |
|-----------------------|--------------------------------------|
| Amazon S3             | 269ba276-16fc-47db-92b0-c1049a3c131f |
| Azure Blob Storage    | 95bd8965-fc8a-4119-b9c3-944c2c2df6d2 |
| Azure Data Lake       | 17be2013-2549-41ce-96e7-a70363bec293 |
| Data Landing Zone     | cd2fc47e-e838-4f38-a581-8fff2f99b63a |
| Google Cloud Storage  | 585c15c4-6cbf-4126-8f87-e26bff78b657 |
| SFTP                  | 354d6aad-4754-46e4-a576-1b384561c440 |

In order to execute the data flow, There are two options available to you:
- If you do not want to wait you can do a **adhoc run** to execute it instantly in Section 4.4.
- Either **wait until it gets scheduled**. We selected to have it run every 3 hours, so you may need to wait up to 3 hours.

We have selected the first option by default, if you would select the second option, please change the boolean on_schedule in below cell to True, skip the step for triggering adhoc run located the first cell in Section 4.4, wait up to 3 hours and execute the cells after.

In [30]:
import time
on_schedule = False
if on_schedule:
    schedule_params = {
        "interval": 3,
        "timeUnit": "hour",
        "startTime": int(time.time())
    }
else:
    schedule_params = {
        "interval": 1,
        "timeUnit": "day",
        "startTime": int(time.time() + 60*60*24*365)
    }


flow_spec_id = "269ba276-16fc-47db-92b0-c1049a3c131f"
flow_obj = {
    "name": f"[CMLE][Week2] Flow for Featurized Dataset to S3 created by {username}",
    "flowSpec": {
        "id": flow_spec_id,
        "version": "1.0"
    },
    "sourceConnectionIds": [
        source_connection_id
    ],
    "targetConnectionIds": [
        target_connection_id
    ],
    "transformations": [],
    "scheduleParams": schedule_params
}

'85ee6381-6c81-4cf7-bdba-097edc8f4bd6'

In [ ]:
flow_res = flow_conn.createFlow(
    obj = flow_obj,
    flow_spec_id = flow_spec_id
)
dataflow_id = flow_res["id"]
dataflow_id

<div class="alert alert-block alert-warning">
<b>Note:</b> 

If you would like to switch to a different cloud storage, you need to update the `flow_spec_id` variable above to the matching value in the table mentioned earlier in this section.
</div>

After you create the data flow, you should be able to see it in the UI to monitor executions, runtimes and its overall lifecycle. You can get the link below and should be able to see it in the UI as shown in the screenshot as well.

In [31]:
dataflow_link = get_ui_link(tenant_id, "destination/browse", dataflow_id)
print(f"Data Flow created as ID {dataflow_id} available under {dataflow_link}")

Data Flow created as ID 85ee6381-6c81-4cf7-bdba-097edc8f4bd6 available under https://experience.adobe.com/#/@aamds/sname:cmle/platform/destination/browse/85ee6381-6c81-4cf7-bdba-097edc8f4bd6


![Dataflow](../media/CMLE-Notebooks-Week2-Dataflow.png)

## 4.4 Executing the Data Flow

At this point we've just created our Data Flow, but it has not executed yet. Please follow the instructions for the option you selected in Section 4.3 :
- If you do not want to wait you can do a **adhoc run** to execute it instantly.
- Either **wait until it gets scheduled**. We selected to have it run every 3 hours, so you may need to wait up to 3 hours.

In the cell below we're showing how to do the first option to trigger a adhoc run, if you selected the second option, you can skip the cell below and will need to wait up to 3 hours to execute the cells after.

<div class="alert alert-block alert-warning">
<b>Note:</b> Please wait at least 10 minutes after creating the dataflow before triggering the next cell, otherwise the job might not execute at all.
</div>

In [35]:
# TODO: use new functionality in aepp when it is released
from aepp import connector

connector = connector.AdobeRequest(
  config_object=aepp.config.config_object,
  header=aepp.config.header,
  loggingEnabled=False,
  logger=None,
)

endpoint = aepp.config.endpoints["global"] + "/data/core/activation/disflowprovider/adhocrun"

payload = {
    "activationInfo": {
        "destinations": [
            {
                "flowId": dataflow_id, 
                "datasets": [
                    {"id": created_dataset_id}
                ]
            }
        ]
    }
}

connector.header.update({"Accept":"application/vnd.adobe.adhoc.dataset.activation+json; version=1"})
activation_res = connector.postData(endpoint=endpoint, data=payload)
activation_res

{'destinations': [{'datasets': [{'id': '642d912512f86b1c07ef5650',
     'statusURL': 'https://platform.adobe.io/data/foundation/flowservice/runs/e4b252c5-607c-4038-a59c-5e71642abe2d',
     'flowId': '85ee6381-6c81-4cf7-bdba-097edc8f4bd6'}]}]}

<div class="alert alert-block alert-warning">
<b>Note:</b> 

If you see an error such as `Invalid parameter: Flow for id 93790efa-645b-4400-8afe-b6f135734656 is incorrect. Error is [Adhoc run can not be executed for Flow spec=cd2fc47e-e838-4f38-a581-8fff2f99b63a.`. it means your cloud storage is not yet whitelisted for exporting datasets. Please reach out to your Adobe contact to have it enabled.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> 

If you see an error such as `Invalid parameter: Following order ID(s) are not ready for dataset export, please wait for 10 minutes and retry.`. it means you need to wait a few minutes and retry again.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> 

If you get a message saying a run already exists, it means that either this dataset has been exported already based on the schedule, or that you've already done an adhoc export before.
</div>

Now we can check the execution of our Data Flow to make sure it actually executes. You can run the following cell until you can see the run appear.

In [39]:
import time

# TODO: handle that more gracefully in aepp
finished = False
while not finished:
    try:
        runs = flow_conn.getRuns(prop=f"flowId=={dataflow_id}")
        for run in runs:
            run_id = run["id"]
            run_started_at = run["metrics"]["durationSummary"]["startedAtUTC"]
            run_ended_at = run["metrics"]["durationSummary"]["completedAtUTC"]
            run_duration_secs = (run_ended_at - run_started_at) / 1000
            run_size_mb = run["metrics"]["sizeSummary"]["outputBytes"] / 1024. / 1024.
            run_num_rows = run["metrics"]["recordSummary"]["outputRecordCount"]
            run_num_files = run["metrics"]["fileSummary"]["outputFileCount"]
            print(f"Run ID {run_id} completed with: duration={run_duration_secs} secs; size={run_size_mb} MB; num_rows={run_num_rows}; num_files={run_num_files}")
        finished = True
    except Exception as e:
        print(f"No runs completed yet for flow {dataflow_id}")
        time.sleep(30)

Run ID e4b252c5-607c-4038-a59c-5e71642abe2d completed with: duration=17.296 secs; size=2.652714729309082 MB; num_rows=89921; num_files=1


## 4.5 Saving the featurized dataset to the configuration

Now that we got everything working, we just need to save the `created_dataset_id` variable in the original configuration file, so we can refer to it in the following weekly assignments. To do that, execute the code below:

In [42]:
config.set("Platform", "featurized_dataset_id", created_dataset_id)

with open(config_path, "w") as configfile:
    config.write(configfile)